In [0]:
%sql
-- Find the top 10 customers who have generated the highest revenue
select *
from
(
  SELECT 
      customer_skey,
      c.CustomerID,
      concat(c.CustomerFirstName,' ',c.CustomerLastName) as customer_name,
      round(SUM(of.Quantity * of.UnitPrice),2) AS total_revenue,
      RANK() OVER (ORDER BY SUM(of.Quantity * of.UnitPrice) DESC) AS rank_customer
  FROM gold.orders_fact of
  LEFT JOIN gold.customers c
      ON c.CustomerID = of.CustomerID
  GROUP BY 
      customer_skey,
      c.CustomerID,
      concat(c.CustomerFirstName,' ',c.CustomerLastName) 
  ORDER BY total_revenue DESC
)
where 
  rank_customer <= 10

customer_skey,CustomerID,customer_name,total_revenue,rank_customer
fe93f6ad-4c8a-4df9-aae4-16050bafd246,85,Summer Torres,207946.96,1
b939b8e5-c135-4868-ae19-2a1ae09f1cbe,561,Dora Moyer,189338.57,2
96133362-5571-4872-bf7e-b7172960f4b8,525,Oliver Lutz,182465.38,3
e16778cb-ce60-4cdc-a4f9-94c3cb2280f4,9675,Justin Huber,180937.14,4
c87098a0-de7d-497b-9f6d-fab2c7f52bd6,383,Gerard Griffin,180225.48,5
5d890760-1844-41c7-b1c1-d6c29bcc56f3,92,Morgan Weber,177734.73,6
2878e752-5033-4ff1-bc31-05adcc8acb27,736,Frank Parker,177066.6,7
b519f2d0-330a-4856-9294-1caf86105cea,257,Mark Gentry,176436.87,8
67c1d375-d5a7-4432-b455-30402493cdc5,32,Ernest Gamble,175872.42,9
db86e7ed-fd56-4240-adc1-e5b863ddcce5,3766,Scott Becker,174214.6,10


%md
## Part-to-Whole Analysis
### Purpose:
    - To compare performance or metrics across dimensions or time periods.
    - To evaluate differences between categories.
    - Useful for A/B testing or regional comparisons.

In [0]:
%sql
-- Which customer segments contribute the most to overall revenue?
WITH segment_revenue AS (
    SELECT
        customer_segment,
        round(SUM(total_revenue),2) AS total_revenue,
        COUNT(total_products) AS total_products,
        round(SUM(avg_monthly_spend),2) AS total_avg_monthly_spend
    FROM gold.customer_segment
    GROUP BY customer_segment
)
SELECT
    customer_segment,
    total_products,
    total_revenue,
    total_avg_monthly_spend,
    ROUND((CAST(total_revenue AS FLOAT) / SUM(total_revenue) OVER ()) * 100, 2) AS percentage_of_total,
    RANK() OVER (ORDER BY total_revenue DESC) AS rank_customer
FROM segment_revenue;

customer_segment,total_products,total_revenue,total_avg_monthly_spend,percentage_of_total,rank_customer
Loyal,8081,4.4639044488E8,8655811.09,87.83,1
New,1348,5.834954906E7,2.540378826E7,11.48,2
Regular,571,3523698.07,68668.96,0.69,3



## RFM analysis (Recency, Frequency, and Monetary)
### Purpose:
    - Customer segmentation technique used to evaluate Recency, Frequency, and Monetary value.
    - Identifying your most valuable customers versus those at risk of churning.

In [0]:
%sql
create view gold.RFM_view as 
WITH RawData AS (
    /*
    Select columns needed for analyses
    */
    SELECT 
        CustomerID,
        DATEDIFF(day, MAX(LastEditedDate), GETDATE()) AS Recency, -- Time since last purchase
        COUNT(OrderID) AS Frequency, -- Total orders
        round(SUM(unitprice * Quantity),2) AS Monetary -- Total spend
    FROM gold.orders_fact
    GROUP BY CustomerID
),
RankedData AS (
    /*
    Assign Scoring (Quintiles) 
      •	Recency: Lower values (more recent) get higher scores (rank by Recency DESC).
      •	Frequency/Monetary: Higher values get higher scores (rank by ASC).
    */
    SELECT 
        CustomerID,
        NTILE(5) OVER (ORDER BY Recency DESC) AS R_Score,
        NTILE(5) OVER (ORDER BY Frequency ASC) AS F_Score,
        NTILE(5) OVER (ORDER BY Monetary ASC) AS M_Score
    from 
      RawData 

)
SELECT 
    /*
      Segment Your Customers
    */
    CustomerID,
    CONCAT(R_Score, F_Score, M_Score) AS RFM_Cell, -- Combined 3-digit score
    CASE 
        WHEN R_Score = 5 AND F_Score = 5 AND M_Score = 5 THEN 'VIP Customers'
        WHEN R_Score >= 4 AND F_Score >= 4 THEN 'Loyal Customers'
        WHEN R_Score = 5 AND F_Score = 1 THEN 'New Customers'
        WHEN R_Score <= 2 THEN 'At Risk'
        ELSE 'Other'
    END AS Customer_Segment
FROM RankedData;



In [0]:
%sql
select  
  Customer_Segment,
  count(1) as customer_count
from gold.RFM_view
group by 
  Customer_Segment

Customer_Segment,customer_count
At Risk,4000
Other,4004
Loyal Customers,1513
New Customers,360
VIP Customers,123


## Migration Rates
### Purpose:
    - Compare a customer's segment at two different points in time (The flow of customers).

In [0]:
%sql
select MAX(LastEditedWhen) AS MaxDate
    from gold.orders_fact

MaxDate
2025-08-31T07:48:59.000Z


In [0]:
%sql

  with base_query as(
    select 
      of.OrderID,
      of.LastEditedWhen,
      c.CustomerID,
      DATEDIFF(day, MAX(of.LastEditedDate), GETDATE()) AS Recency
    from 
      gold.dim_date dd
    join
      gold.orders_fact of 
    on of.LastEditedDate = dd.date
    left join 
      gold.customers c 
    on of.CustomerID = c.CustomerID
    where 
      c.active_flg = 1
    group by 
      of.OrderID,
      of.LastEditedWhen,
      c.CustomerID
  ), customer_orders as (
  select 
    CustomerID,
    Recency,
    count(distinct OrderID) as total_orders,
    max(LastEditedWhen) as last_order_date
  from 
    base_query
  group by 
    CustomerID,
    Recency
  ),
  MaxDateCTE as (
    select MAX(LastEditedWhen) AS MaxDate
    from gold.orders_fact
    where year(LastEditedWhen) = 2019
  )
  ,period_1 as (
    select c.*
    from 
      customer_orders c
    cross join 
      MaxDateCTE M
    where c.last_order_date >= DATEADD(month, -6, M.MaxDate) and c.last_order_date < DATEADD(month, -3, M.MaxDate)
   ),
   period_2 as (
    select c.*
    from 
      customer_orders c
    cross join 
      MaxDateCTE M
    where c.last_order_date >= DATEADD(month, -3, M.MaxDate) and c.last_order_date <= M.MaxDate 
   ),
   Segment_History AS (
    SELECT 
        coalesce(p1.CustomerID, p2.CustomerID) AS CustomerID,
        -- Period 1: e.g., 6 months ago to 3 months ago
        CASE 
            WHEN p1.recency <= 30 AND p1.Total_Orders > 5 THEN 'Loyal'
            WHEN p1.recency > 90 THEN 'At Risk'
            ELSE 'Standard'
        END AS Segment_P1,
        -- Period 2: e.g., the last 3 months
        CASE 
        WHEN p2.recency <= 30 AND p2.Total_Orders > 5 THEN 'Loyal'
            WHEN p2.recency > 90 THEN 'At Risk'
            ELSE 'Standard'
        END AS Segment_P2
    FROM period_1 AS p1
    FULL JOIN period_2 AS p2 ON p1.CustomerID = p2.CustomerID
),
Migration_Counts AS (
    SELECT 
        Segment_P1, 
        Segment_P2, 
        COUNT(CustomerID) AS Customer_Count
    FROM Segment_History
    GROUP BY Segment_P1, Segment_P2
)
SELECT 
    Segment_P1 AS Starting_Segment,
    Segment_P2 AS Ending_Segment,
    Customer_Count,
    -- Calculation: (Moved to B) / (Total who started in A)
    CAST(Customer_Count AS FLOAT) / 
        SUM(Customer_Count) OVER(PARTITION BY Segment_P1) * 100 AS Migration_Rate_Pct
FROM Migration_Counts
ORDER BY Starting_Segment, Migration_Rate_Pct DESC

;;

Starting_Segment,Ending_Segment,Customer_Count,Migration_Rate_Pct
At Risk,Standard,242,98.3739837398374
At Risk,At Risk,4,1.6260162601626018
Standard,At Risk,275,100.0


## Predictive Customer Lifetime Value (CLV) forecasting 
### Purpose:
    - Calculate the expected future revenue a customer will generate based on their past behavior.

In [0]:
WITH Customer_Base AS (
    SELECT 
      /*
      Select main columns for analyses
      */
        CustomerID,
        MIN(LastEditedWhen) as first_purchase,
        MAX(LastEditedWhen) as last_purchase,
        COUNT(DISTINCT OrderID) as total_orders,
        round(SUM(UnitPrice * Quantity),2) as total_revenue,
        round(AVG(UnitPrice * Quantity),2) as avg_order_value -- Average Purchase Value (APV)
    FROM gold.orders_fact
    GROUP BY CustomerID
),
Ratios AS (
  /*
    Calculate Key Prediction Ratios
    -	APFR: How many times they buy in a given time unit (e.g., per year).
    -	Lifespan: The duration in years between their first and most recent purchase

    NB To forecast, you need the Average Purchase Frequency Rate (APFR) and the Average Customer Lifespan (ACL). 

  */
    SELECT 
        CustomerID,
        avg_order_value,
        -- Frequency: orders per year
        (CAST(total_orders AS FLOAT) / 
         NULLIF(DATEDIFF(day, first_purchase, last_purchase), 0) * 365) AS annual_frequency,
        -- Lifespan: total years active
        DATEDIFF(day, first_purchase, last_purchase) / 365.0 AS current_lifespan_years
    FROM Customer_Base
)
SELECT 
  /*
    Forecast Future CLV
  */
    CustomerID,
    -- Simple Forecast: (Avg Order Value * Annual Frequency) * Predicted Future Years
    (avg_order_value * annual_frequency) * 3 AS predicted_3yr_clv,
    
    -- Discounted Forecast: accounting for a 10% annual churn probability
    (avg_order_value * annual_frequency) * (1 - POWER(0.9, 3)) / (1 - 0.9) AS discounted_3yr_clv
FROM Ratios;




CustomerID,predicted_3yr_clv,discounted_3yr_clv
604,3434.596468123317,3102.585476204729
877,30808.57015974441,27830.40837763577
321,24295.181770088646,21946.6475323134
671,79605.97744034708,71910.73295444684
600,7965.191961414791,7195.223405144691
626,4934.686432706222,4457.666744211286
160,18188.443439363815,16430.227240225307
158,16163.12312324368,14600.687887996786
717,35022.86628733997,31637.32254623043
312,25897.9414507772,23394.47377720206


## Refined Lifetime Value (CLV) forecasts
### Adjust the "predicted future years" based on the probability that a customer is still active
### A Churn Risk Score acts as a multiplier: if the risk is high (e.g., 90%), your CLV forecast should drop significantly

In [0]:
WITH Customer_Behavior AS (
  /*
    Identify Risk Indicators
    We calculate indicators that signal a fading relationship, specifically Recency vs. Average Gap. If a customer usually buys every 30 days but hasn't bought in 60, their risk is rising

  */
    SELECT 
      /*
      Calculate key indicators
      */
        CustomerID,
        DATEDIFF(day, MAX(LastEditedWhen), GETDATE()) AS days_since_last_purchase,
        COUNT(OrderID) AS total_orders,
        -- Calculate the average time between purchases (Inter-purchase Time)
        DATEDIFF(day, MIN(LastEditedWhen), MAX(LastEditedWhen)) / NULLIF(COUNT(OrderID) - 1, 0) AS avg_purchase_gap
    FROM gold.orders_fact
    GROUP BY CustomerID
),
Risk_Scoring AS (
  /*
    Calculate the Churn Risk Score
    We use a ratio of Current Silence / Typical Gap.
    -	Score 0-1.0: Behavior is normal.
    -	Score > 1.5: Customer is "overdue" and likely drifting.
    -	Score > 3.0: High probability of churn.

  */
    SELECT 
        *,
        CASE 
            WHEN avg_purchase_gap IS NULL THEN 0.5 -- New customers (neutral risk)
            ELSE (CAST(days_since_last_purchase AS FLOAT) / NULLIF(avg_purchase_gap, 0))
        END AS deviation_ratio
    FROM Customer_Behavior
)
SELECT 
/*
  Refine the CLV Forecast
  - Integrate the risk score into your prediction
*/
    CustomerID,
    deviation_ratio,
    CASE 
        WHEN deviation_ratio < 1.0 THEN 'Low'
        WHEN deviation_ratio BETWEEN 1.0 AND 2.0 THEN 'Medium'
        ELSE 'High'
    END AS risk_category,
    -- Survival Probability: The higher the deviation, the lower the probability
    CASE 
        WHEN deviation_ratio <= 1.0 THEN 1.0  -- 100% likely to stay
        WHEN deviation_ratio > 3.0 THEN 0.1   -- 10% likely to stay
        ELSE (1 / deviation_ratio)            -- Sliding scale
    END AS survival_prob,
    -- Refined CLV: (Historical Annual Value) * Survival Probability
    (total_orders * survival_prob) AS adjusted_expected_orders_yr
FROM Risk_Scoring;
/*
Note
Why this Refines Forecasts
-	Avoids Over-Optimism: Without this, a "Champion" who hasn't bought in 2 years would still show a massive predicted CLV.
-	Triggers Intervention: You can export customers where risk_category = 'Medium' directly to a Marketing Automation tool like Braze for a "Win-back" campaign.
-	Dynamic Budgeting: It allows finance teams to see a "Risk-Adjusted" revenue pipeline.


*/


CustomerID,deviation_ratio,risk_category,survival_prob,adjusted_expected_orders_yr
604,0.024693205627057766,Low,1.0,2.0
877,0.9373801916932908,Low,1.0,2.0
321,2.905919359450958,High,0.34412517221019484,1.0323755166305846
671,4.028922631959508,High,0.1,0.30000000000000004
600,1.0207216863165416,Medium,0.9796989849492475,1.959397969898495
626,1.0785094066570189,Medium,0.9272056356927205,1.854411271385441
160,1.5542301745085045,Medium,0.6434053439454235,1.9302160318362704
158,1.3083099156965075,Medium,0.7643448910708807,1.5286897821417613
717,3.220483641536273,High,0.1,0.2
312,3.0738341968911915,High,0.1,0.2
